## お笑い（特に漫才）のテキストデータ分析

### 問い

1. 関西と関東で言葉の言い回しが違う？
    1. 関西は単純な言葉とノリで、関東はひねった言い回し？
2. 各芸人のネタを共起ネットワークに起こしてみると、芸人によって何か特徴が見られるか
3. ...

漫才のデータ分析をした記事もあったので参考にする。
感情分析はちょっと面白そう
- [データ分析で、M-1グランプリの"高得点パターン"を探してみた](https://note.com/natto_nebaneba/n/n11b4229f8b19)


### データの用意

ネタ台本の書き起こしをしているサイトからとってくる

- https://himapucchi.com/
- https://comedy.shivatax.com/category/manzaiwrite/
- https://manzaidaihon.com/
- https://bukiyo30.hateblo.jp/
- https://waraitext.com/
- https://hansode5.com/category/comedy-english/manzai/




# Code

## スクレイピング

https://himapucchi.com/
のスクレイピング

In [3]:
import os
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [29]:
# フォルダ
folder_name_raw = './data/raw'
print(os.path.exists(folder_name_raw))

folder_name_tidy = "./data/tidy"
print(os.path.exists(folder_name_tidy))

True
True


In [21]:
root_url = "https://himapucchi.com/"

In [22]:
# 17ページに分かれているのでlistを作成

crawl_list = []
crawl_list.append(root_url)

for i in range(16):
    crawl_list.append(f"{root_url}page/{i+2}")

crawl_list

['https://himapucchi.com/',
 'https://himapucchi.com/page/2',
 'https://himapucchi.com/page/3',
 'https://himapucchi.com/page/4',
 'https://himapucchi.com/page/5',
 'https://himapucchi.com/page/6',
 'https://himapucchi.com/page/7',
 'https://himapucchi.com/page/8',
 'https://himapucchi.com/page/9',
 'https://himapucchi.com/page/10',
 'https://himapucchi.com/page/11',
 'https://himapucchi.com/page/12',
 'https://himapucchi.com/page/13',
 'https://himapucchi.com/page/14',
 'https://himapucchi.com/page/15',
 'https://himapucchi.com/page/16',
 'https://himapucchi.com/page/17']

In [23]:
# <a>のlistを引数に、<a>内のURLとtitleを取得しdict listを返す
def extract_url_title_himapucchi(a_list):
    page_url_title = []
    for content in a_list:
        url = content.get('href')  # href属性を取得
        title = content.get("title")  # titleを取得
        page_url_title.append({"page_url": url, "title": title})
    return page_url_title


In [24]:
# User-Agent の設定
dummy_user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'


In [27]:
# htmlリクエスト & soup

page_url_title = []

for crawl_url in crawl_list:
    response = requests.get(crawl_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")

    page_list = soup.select("#main > #list > a")
    page_url_title.extend(extract_url_title_himapucchi(page_list))

    time.sleep(3) # 負荷軽減


In [28]:
page_df = pd.DataFrame(page_url_title)
page_df

,page_url,title
0,https://himapucchi.com/m2023-final-kurage/,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」
1,https://himapucchi.com/m2023-final-danbira/,【M-1グランプリ2023 1stラウンド】ダンビラムーチョ「カラオケボックス」
2,https://himapucchi.com/m2023-final-shinkujeshika/,【M-1グランプリ2023 1stラウンド】真空ジェシカ「B画館」
3,https://himapucchi.com/m2023-final-mayurika/,【M-1グランプリ2023 1stラウンド】マユリカ「倦怠期」
4,https://himapucchi.com/m2023-final-kabeposutar/,【M-1グランプリ2023 1stラウンド】カベポスター「おまじない」
...,...,...
163,https://himapucchi.com/m2019first-milkboy/,【M-1グランプリ2019 1stラウンド】<br>ミルクボーイ「コーンフレーク」
164,https://himapucchi.com/m2019first-kamaitachi/,【M-1グランプリ2019 1stラウンド】<br>かまいたち「UFJ」
165,https://himapucchi.com/m2019final-milkboy/,【M-1グランプリ2019 最終決戦】<br>ミルクボーイ「最中」
166,https://himapucchi.com/m2019final-kamaitachi/,【M-1グランプリ2019 最終決戦】<br>かまいたち「トトロ」


In [31]:
# いったんcsvに保存
page_df.to_csv(os.path.join(folder_name_raw, "page_url_title.csv"), index=False)